In [2]:
%matplotlib inline
import pandas as pd


<img src="header.png" alt="drawing"/>


A multi-channel electroencephalography (EEG) system enables a broad range of applications including neurotherapy, biofeedback, and brain computer interfacing. It has 14 EEG channels with names based on the International 10-20 locations: AF3, F7, F3, FC5, T7, P7, O1, O2, P8, T8, FC6, F4, F8, AF4:

<img src="EEG.png" alt="drawing" width="200"/>

The dataset you will analyse is created with the [Emotiv EPOC+](https://www.emotiv.com/product/emotiv-epoc-14-channel-mobile-eeg). All data is from one continuous EEG measurement with the Emotiv EEG Neuroheadset. The duration of the measurement was 117 seconds. The eye state was detected via a camera during the EEG measurement and later added manually to the file after analyzing the video frames. A '1' indicates the eye-closed and '0' the eye-open state.(*Source: Oliver Roesler, Stuttgart, Germany*)

Let's load the train and test set:

In [3]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")
testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")
sample_submission = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/sample_submission.csv")

HTTPError: HTTP Error 500: Internal Server Error

Load the test set file into a Pandas DataFrame called `test`. 

In [ ]:
#solution

Let's look at the first 5 rows of the train set.

In [ ]:
train.head(5)

We see 14 columns that each correspond to the measurements of one EEG channel. Each row is a time-point. The column `eyeDetection` annotates each time-point with '1' when the eye was recorded as closed and '0' when the eye was recorded as open.  

Our goal is to build a predictive model from the train set that is able to accuratly predict the eye states in the test set from the 14 channel measurements. 

Show the last 10 rows of the test set:

In [ ]:
#solution

This is a blind test so the `eyeDetection` column is not available in the test set. The test set does contain a column called `index` that you will need to send your predictions to the Kaggle website.

Let's look at the shape of the train and test set:

In [ ]:
print(train.shape)
print(test.shape)

How many time-points does the test set contain?

We can get some general statistics about the train set using the DataFrame `.describe()` function:

In [ ]:
train.describe().transpose()

# 1. Building a decision tree

The scikit-learn `DecisionTreeClassifier` class computes a decision tree predictive model from a data set. To get all the options for learning you can simply type: 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
help(DecisionTreeClassifier)

Let's create a decision tree model:

In [ ]:
cls_DT = DecisionTreeClassifier()

This creates a decision tree model with default values for the hyperparameters:

In [ ]:
cls_DT

To fit the train set we need to have our features and labels in different DataFrames. I call them `X` and `y`:

In [ ]:
X = train.sample(frac=1).copy()
y = X.pop("eyeDetection")

print(X.shape)
print(y.shape)

Now we can fit the train set in just one line of code:

In [ ]:
cls_DT.fit(X,y)

Let's see how well the decision tree performs on the train set:

In [ ]:
from sklearn.metrics import accuracy_score

predictions = cls_DT.predict(X)
print("Accuracy: %f"%(accuracy_score(predictions, y)))

Great! With just a few lines of code a get perfect accuracy on my data set!

These are the predictions for the first 20 data points:

In [ ]:
predictions[0:20]

The following code plots the fitted decision tree `cls` as a `tree.png` file:

In [ ]:
from sklearn import tree
from io import StringIO
from IPython.display import Image, display
import pydotplus

out = StringIO()
tree.export_graphviz(cls_DT, out_file=out)
graph=pydotplus.graph_from_dot_data(out.getvalue())
graph.write_png("tree.png")

# 2. Kaggle evaluation

Let's make a Kaggle submission:

In [ ]:
#code for submission
index_col = test.pop("index")
predictions_test = cls_DT.predict(test)
out_tmp = pd.DataFrame()
out_tmp["index"] = index_col
out_tmp["eyeDetection"] = predictions_test
out_tmp.to_csv("submission.csv",index=False)

# 3. The test set

Since we don't have the test set `eyeDetection` labels we cannot check overfitting on the test set before submitting our predictions. So we need to create our own test set (**not seen during training!**) with known class labels.

Scikit-learn offers many options to do this. One of them is the `train_test_split` function:

In [ ]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X,y,test_size=.2, random_state=random_seed)

#train fold
print(train_X.shape)
print(train_y.shape)
#validation fold
print(val_X.shape)
print(val_y.shape)

Fit a decision tree model with default paramters on the `train_X` data set.

In [ ]:
#solution

What is the accuracy on `train_X`? 

In [ ]:
#solution

What is the accuracy on `val_X`?

In [ ]:
#solution

What do you see?


# 4. Hyperparameters

Let's look at the  `DecisionTreeClassifier` again.

In [ ]:
help(DecisionTreeClassifier)

Notice that there are many parameters (**hyperparameters**) that influence the contruction of a decision tree. One of them is `max_depth` that limits the depth of the contructed decicison tree. The following code evaluates different values for this hyperparameter.

In [ ]:
for maxdepth in range(3,24,1):
    cls = DecisionTreeClassifier(max_depth=maxdepth)
    cls.fit(train_X,train_y)
    predictions_train = cls.predict(train_X)
    predictions_val = cls.predict(val_X)
    print("%i %f %f"%(maxdepth,accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

Construct a decision tree with `max_depth` equal to `None` and print the first 20 predictions on the train set. 

In [ ]:
#solution

These are binary class predictions (0 or 1). Sometimes it makes sense to predict the probalbility of a data point to belong to a certain class (typically class 1 which is also known as the positive class).

In [ ]:
predictions_train = cls_DT.predict_proba(train_X)
print(predictions_train[0:20])

The frist column shows the probability of a data point to belong to class 0 and the second column shows the probability of a data point to belong to class 1. What do you see? Why is this?

Can you contruct a decision tree that does not just output 0 or 1 probabilities?

In [ ]:
#solution

Play with the hyperparameters in a Template notebook and make some Kaggle submissions.

# 5. Ensemble learning: bagging

We have seen that bias and variance play an important role in Machine Learning. Let's first see what bagging can do for our data set. 

In [ ]:
from sklearn.ensemble import BaggingClassifier

cls_bagged = BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=random_seed)
                                                            
cls_bagged.fit(train_X,train_y)
predictions_train = cls_bagged.predict(train_X)
predictions_val = cls_bagged.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

With the `RandomForestClassifier` the variance of the decision tree is reduced also by selecting features for decision tree contruction at random. Let's see how far we get with default hyperparameter values.   

In [ ]:
from sklearn.ensemble import RandomForestClassifier

cls_RF = RandomForestClassifier(random_state=random_seed)
cls_RF.fit(train_X,train_y)
predictions_train = cls_RF.predict(train_X)
predictions_val = cls_RF.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

Now let's see what we can do with the `RandomForestClassifier`.

In [ ]:
help(RandomForestClassifier)

What happens when you increase the `n_estimators` hyperparameter? 

In [ ]:
#solution

# 6. Ensemble learning: boosting

How about the `GradientBoostingClassifier`?

In [ ]:
#solution
from sklearn.ensemble import GradientBoostingClassifier

cls_GB = GradientBoostingClassifier(random_state=random_seed,max_depth=10)
cls_GB.fit(train_X,train_y)
predictions_train = cls_GB.predict(train_X)
predictions_val = cls_GB.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

# 7. Logistic regression

Before we evaluate other Machine Learning algorithms such as logistic regression we need to look at the distributions and scales of the features. In fact, this should always be the first thing to do when analyzing a new data set.

In Pandas we can create a boxplot for each of the features as follows:

In [ ]:
X.boxplot()

What do we see?

There are many approaches to outlier detection and removal. Let's use a simple one:

In [ ]:
from scipy import stats

print(X.shape)
tmp = (np.abs(stats.zscore(X)) < 3).all(axis=1)
X_clean = X[tmp]
y_clean = y[tmp]
print(X_clean.shape)

In [ ]:
X_clean.boxplot()

This should look much better.

However, feature values between features are still at very different scales. We need to normalize them. 

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(X_clean)
X_norm = pd.DataFrame(scaler.transform(X_clean),columns=X_clean.columns)

In [ ]:
X_norm.boxplot()

Now each feature is at a similar scale.

Let's start with `LogisticRegression`.

In [ ]:
from sklearn.linear_model import LogisticRegression

train_X, val_X, train_y, val_y = train_test_split(X_norm,y_clean,test_size=.2, random_state=random_seed)

cls_LR=LogisticRegression()
cls_LR.fit(train_X,train_y)
predictions_train = cls_LR.predict(train_X)
predictions_val = cls_LR.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

Does this perform well?

What could be the reason?

# 8. Non-linear transformations

What does linear classification mean?

In [ ]:
import seaborn as sns

dataset_tmp = pd.read_csv('svm_example1.csv')

sns.lmplot(x="x1", y="x2", data=dataset_tmp, hue='y', markers=['o','+'], 
           fit_reg=False, height=5, scatter_kws={"s": 80})

In [ ]:
import imp
compomics_import = imp.load_source('compomics_import', 'compomics_import.py')

X_tmp = dataset_tmp.copy()
y_tmp = X_tmp.pop('y')

cls_LR.fit(X_tmp,y_tmp)

sns.lmplot(x="x1", y="x2", data=dataset_tmp, hue='y', markers=['o','+'], 
           fit_reg=False, height=5, scatter_kws={"s": 80})
compomics_import.plot_svm_decision_function(cls_LR)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

polynomial_features = PolynomialFeatures(degree=2)
model = Pipeline([("polynomial_features", polynomial_features),
                         ("LR", cls_LR)])
model.fit(X_tmp,y_tmp)

sns.lmplot(x="x1", y="x2", data=dataset_tmp, hue='y', markers=['o','+'], 
           fit_reg=False, height=5, scatter_kws={"s": 80})
compomics_import.plot_svm_decision_function(model)

What happens when you increase the degree?

In [ ]:
#solution

Would this work on our dataset?

In [ ]:
#solution 

Kaggle submission?

The following code ranks the features based on the value of the hyperparameters:

In [ ]:
indices = np.argsort(cls_LR.coef_[0])[::-1]
fnames = poly.get_feature_names()
for i in range(len(fnames)):
    print("%f\t%s"%(cls_LR.coef_[0][indices[i]],fnames[indices[i]]))


What is wrong with this code?

In [ ]:
#solution

# 9. Support Vector Machines

Let's see what an SVM can do with an RBF kernel:

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='poly',C=1,degree=2)
model.fit(X_tmp,y_tmp)

sns.lmplot(x="x1", y="x2", data=dataset_tmp, hue='y', markers=['o','+'], 
           fit_reg=False, height=5, scatter_kws={"s": 80})
compomics_import.plot_svm_decision_function(model)

In [ ]:
dataset2_tmp = pd.read_csv("svm_example2.csv")
X_tmp = dataset2_tmp.copy()
y_tmp = X_tmp.pop('y')

model = SVC(kernel='rbf',C=1,gamma=1)
model.fit(X_tmp,y_tmp)

sns.lmplot(x="x1", y="x2", data=dataset2_tmp, hue='y', markers=['o','+'], 
           fit_reg=False, height=5, scatter_kws={"s": 80})
compomics_import.plot_svm_decision_function(model)

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X_norm,y_clean,test_size=.2, random_state=random_seed)

cls_SVM = SVC(kernel='rbf')
cls_SVM.fit(train_X,train_y)
predictions_train = cls_SVM.predict(train_X)
predictions_val = cls_SVM.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

# 10. Neural Networks

How about a Neural Network?

In [ ]:
from sklearn.neural_network import MLPClassifier

cls_MLP=MLPClassifier(hidden_layer_sizes=(60,10))
cls_MLP.fit(train_X,train_y)
predictions_train = cls_MLP.predict(train_X)
predictions_val = cls_MLP.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

Is feature normalization required?

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X,y,test_size=.2, random_state=random_seed)
cls_MLP.fit(train_X,train_y)
predictions_train = cls_MLP.predict(train_X)
predictions_val = cls_MLP.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))